# Anomaly Detection Lab — Grand Marina Water System

This notebook trains and evaluates anomaly detection models on synthetic IoT sensor data
matching the Grand Marina's water monitoring pipeline.

**How to use this notebook:**
1. Run all cells from top to bottom (`Runtime > Run all`)
2. Observe the outputs — data generation, model training, and visualizations
3. When you see `# === YOUR EXPERIMENT ===`, that's where you change values
4. Record your results in the tables provided in the task instructions

**You do not need to write code from scratch.** Everything is built for you.
Your job is to run experiments, observe results, and draw conclusions.

---
## Section 1: Setup and Data Generation

This section installs required libraries and generates synthetic sensor data
that matches the ranges used by `publisher_defended.py` in Project 6.

In [ ]:
# Install required libraries (already available in Colab, but just in case)
!pip install -q scikit-learn matplotlib numpy pandas joblib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")

In [ ]:
# =============================================================================
# Generate Synthetic Sensor Data
# These ranges match publisher_defended.py from Project 6
# =============================================================================

N_NORMAL = 1000    # Normal sensor readings
N_ANOMALY = 50     # Injected anomalies

# --- Normal data ---
# Pressure: 58-62 PSI (matches publisher_defended.py)
normal_pressure = np.random.uniform(58.0, 62.0, N_NORMAL)

# Flow rate: 45-55 LPM, correlated with pressure (matches publisher_defended.py)
normal_flow = normal_pressure * 0.85 + np.random.normal(0, 1.5, N_NORMAL)
normal_flow = np.clip(normal_flow, 45.0, 55.0)

# Gate position: 42-48% (matches publisher_defended.py)
normal_gate = np.random.uniform(42.0, 48.0, N_NORMAL)

# --- Anomalous data (5 types) ---
n_each = N_ANOMALY // 5

# Type 1: Pressure spikes (>70 PSI — clearly outside 58-62 range)
anom_pressure_1 = np.random.uniform(70.0, 85.0, n_each)
anom_flow_1 = np.random.uniform(48.0, 55.0, n_each)  # normal flow
anom_gate_1 = np.random.uniform(42.0, 48.0, n_each)

# Type 2: Flow drops (<30 LPM — clearly outside 45-55 range)
anom_pressure_2 = np.random.uniform(58.0, 62.0, n_each)  # normal pressure
anom_flow_2 = np.random.uniform(15.0, 28.0, n_each)  # abnormally low
anom_gate_2 = np.random.uniform(42.0, 48.0, n_each)

# Type 3: Stuck sensors (identical readings — same value every time)
anom_pressure_3 = np.full(n_each, 60.00)  # inside normal range but suspiciously constant
anom_flow_3 = np.full(n_each, 51.00)
anom_gate_3 = np.full(n_each, 45.00)

# Type 4: Correlation breaks (high pressure + low flow — physically unlikely)
anom_pressure_4 = np.random.uniform(68.0, 78.0, n_each)
anom_flow_4 = np.random.uniform(15.0, 25.0, n_each)
anom_gate_4 = np.random.uniform(42.0, 48.0, n_each)

# Type 5: Slow drift (pressure climbing steadily from 62 up to 80)
anom_pressure_5 = np.linspace(62.0, 80.0, n_each)
anom_flow_5 = np.random.uniform(48.0, 55.0, n_each)
anom_gate_5 = np.random.uniform(42.0, 48.0, n_each)

# Combine all data
all_pressure = np.concatenate([normal_pressure, anom_pressure_1, anom_pressure_2,
                                anom_pressure_3, anom_pressure_4, anom_pressure_5])
all_flow = np.concatenate([normal_flow, anom_flow_1, anom_flow_2,
                           anom_flow_3, anom_flow_4, anom_flow_5])
all_gate = np.concatenate([normal_gate, anom_gate_1, anom_gate_2,
                           anom_gate_3, anom_gate_4, anom_gate_5])

# Labels: 1 = normal, -1 = anomaly (matching sklearn convention)
labels = np.concatenate([np.ones(N_NORMAL), -np.ones(N_ANOMALY)])

# Create feature matrix
X = np.column_stack([all_pressure, all_flow, all_gate])

# Shuffle the data
shuffle_idx = np.random.permutation(len(X))
X = X[shuffle_idx]
labels = labels[shuffle_idx]

# Create a DataFrame for easier inspection
df = pd.DataFrame(X, columns=['pressure_psi', 'flow_rate_lpm', 'gate_position_pct'])
df['is_anomaly'] = (labels == -1)

print(f"Dataset created:")
print(f"  Total points:   {len(X)}")
print(f"  Normal points:  {(labels == 1).sum()}")
print(f"  Anomaly points: {(labels == -1).sum()}")
print(f"\nFeature ranges:")
print(f"  Pressure: {X[:, 0].min():.1f} - {X[:, 0].max():.1f} PSI")
print(f"  Flow:     {X[:, 1].min():.1f} - {X[:, 1].max():.1f} LPM")
print(f"  Gate:     {X[:, 2].min():.1f} - {X[:, 2].max():.1f} %")
print(f"\nFirst 5 rows:")
df.head()

---
## Section 2: Baseline Isolation Forest Model

Train a baseline Isolation Forest with default-ish parameters and see how it performs.

In [ ]:
# =============================================================================
# Train Baseline Isolation Forest
# =============================================================================

# contamination = expected fraction of anomalies in the data
# We know we injected ~5% anomalies (50 out of 1050)
model = IsolationForest(contamination=0.05, random_state=42)

# Train the model
start_time = time.time()
model.fit(X)
train_time = time.time() - start_time

# Get predictions: 1 = normal, -1 = anomaly
predictions = model.predict(X)

# Get anomaly scores (lower = more anomalous)
scores = model.decision_function(X)

print(f"Baseline Isolation Forest trained in {train_time:.3f} seconds")
print(f"\nPredictions:")
print(f"  Classified as normal:  {(predictions == 1).sum()}")
print(f"  Classified as anomaly: {(predictions == -1).sum()}")

In [ ]:
# =============================================================================
# Evaluate Baseline Results
# =============================================================================

def evaluate_model(true_labels, pred_labels, model_name="Model"):
    """Calculate and print precision, recall, F1 for anomaly detection."""
    # Convert to binary: anomaly=1, normal=0 (for sklearn metrics)
    true_binary = (true_labels == -1).astype(int)
    pred_binary = (pred_labels == -1).astype(int)

    tp = ((pred_binary == 1) & (true_binary == 1)).sum()
    fp = ((pred_binary == 1) & (true_binary == 0)).sum()
    fn = ((pred_binary == 0) & (true_binary == 1)).sum()
    tn = ((pred_binary == 0) & (true_binary == 0)).sum()

    precision = precision_score(true_binary, pred_binary, zero_division=0)
    recall = recall_score(true_binary, pred_binary, zero_division=0)
    f1 = f1_score(true_binary, pred_binary, zero_division=0)

    print(f"\n{'='*50}")
    print(f"  {model_name} Results")
    print(f"{'='*50}")
    print(f"  True Positives  (correctly flagged):   {tp}")
    print(f"  False Positives (normal flagged):      {fp}")
    print(f"  False Negatives (anomalies missed):    {fn}")
    print(f"  True Negatives  (correctly passed):    {tn}")
    print(f"  ---")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  F1 Score:  {f1:.3f}")
    print(f"{'='*50}")

    return {"precision": precision, "recall": recall, "f1": f1,
            "tp": tp, "fp": fp, "fn": fn, "tn": tn}

baseline_results = evaluate_model(labels, predictions, "Baseline Isolation Forest")

---
## Section 3: Visualization

Scatter plot showing normal data (blue) and detected anomalies (red).

In [ ]:
# =============================================================================
# Scatter Plot: Pressure vs. Flow Rate
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Ground truth (what we know)
ax1 = axes[0]
normal_mask = labels == 1
anomaly_mask = labels == -1

ax1.scatter(X[normal_mask, 0], X[normal_mask, 1], c='steelblue', s=10, alpha=0.5, label='Normal')
ax1.scatter(X[anomaly_mask, 0], X[anomaly_mask, 1], c='red', s=30, alpha=0.8, label='Actual Anomaly', marker='x')
ax1.set_xlabel('Pressure (PSI)')
ax1.set_ylabel('Flow Rate (LPM)')
ax1.set_title('Ground Truth: Actual Anomalies')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Model predictions (what the model sees)
ax2 = axes[1]
pred_normal = predictions == 1
pred_anomaly = predictions == -1

ax2.scatter(X[pred_normal, 0], X[pred_normal, 1], c='steelblue', s=10, alpha=0.5, label='Predicted Normal')
ax2.scatter(X[pred_anomaly, 0], X[pred_anomaly, 1], c='red', s=30, alpha=0.8, label='Predicted Anomaly', marker='x')
ax2.set_xlabel('Pressure (PSI)')
ax2.set_ylabel('Flow Rate (LPM)')
ax2.set_title('Model Predictions: Detected Anomalies')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Left plot: where we actually put anomalies")
print("Right plot: where the model thinks anomalies are")
print("Compare the two — how well does the model match reality?")

In [ ]:
# =============================================================================
# Anomaly Score Distribution
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(scores[normal_mask], bins=40, alpha=0.6, color='steelblue', label='Normal data')
ax.hist(scores[anomaly_mask], bins=20, alpha=0.6, color='red', label='Actual anomalies')
ax.axvline(x=0, color='black', linestyle='--', label='Decision boundary')
ax.set_xlabel('Anomaly Score (lower = more anomalous)')
ax.set_ylabel('Count')
ax.set_title('Distribution of Anomaly Scores')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Points to the LEFT of the dashed line are classified as anomalies.")
print("Ideally, all red bars should be left of the line and all blue bars right of it.")

---
## Section 4: Experiment — Contamination Sweep

How does changing the `contamination` parameter affect detection performance?

`contamination` tells the model what fraction of data is anomalous.
Higher values = flag more data as anomalous.

In [ ]:
# === YOUR EXPERIMENT ===
# Try different contamination values and observe how results change.
# The default is 0.05. Try values from very conservative (0.01) to aggressive (0.2).

contamination_values = [0.01, 0.03, 0.05, 0.1, 0.2]

# =============================================================================
# Run the experiment (no changes needed below this line)
# =============================================================================
print(f"{'Contamination':>14} | {'Detected':>8} | {'Precision':>9} | {'Recall':>6} | {'F1':>6}")
print("-" * 60)

contamination_results = {}

for c in contamination_values:
    m = IsolationForest(contamination=c, random_state=42)
    m.fit(X)
    preds = m.predict(X)

    true_bin = (labels == -1).astype(int)
    pred_bin = (preds == -1).astype(int)

    p = precision_score(true_bin, pred_bin, zero_division=0)
    r = recall_score(true_bin, pred_bin, zero_division=0)
    f = f1_score(true_bin, pred_bin, zero_division=0)
    detected = (preds == -1).sum()

    contamination_results[c] = {"precision": p, "recall": r, "f1": f, "detected": detected}

    print(f"{c:>14.2f} | {detected:>8} | {p:>9.3f} | {r:>6.3f} | {f:>6.3f}")

# Find best F1
best_c = max(contamination_results, key=lambda k: contamination_results[k]['f1'])
print(f"\nBest contamination by F1 score: {best_c} (F1 = {contamination_results[best_c]['f1']:.3f})")

---
## Section 5: Experiment — Number of Trees (n_estimators)

More trees = more stable results, but slower training.
At what point do more trees stop helping?

In [ ]:
# === YOUR EXPERIMENT ===
# Try different numbers of trees.

n_estimators_values = [50, 100, 200, 500]

# =============================================================================
# Run the experiment
# =============================================================================
print(f"{'n_estimators':>12} | {'Precision':>9} | {'Recall':>6} | {'F1':>6} | {'Train Time':>10}")
print("-" * 60)

estimator_results = {}

for n in n_estimators_values:
    m = IsolationForest(contamination=0.05, n_estimators=n, random_state=42)

    start = time.time()
    m.fit(X)
    t = time.time() - start

    preds = m.predict(X)
    true_bin = (labels == -1).astype(int)
    pred_bin = (preds == -1).astype(int)

    p = precision_score(true_bin, pred_bin, zero_division=0)
    r = recall_score(true_bin, pred_bin, zero_division=0)
    f = f1_score(true_bin, pred_bin, zero_division=0)

    estimator_results[n] = {"precision": p, "recall": r, "f1": f, "time": t}

    print(f"{n:>12} | {p:>9.3f} | {r:>6.3f} | {f:>6.3f} | {t:>9.3f}s")

best_n = max(estimator_results, key=lambda k: estimator_results[k]['f1'])
print(f"\nBest n_estimators by F1 score: {best_n} (F1 = {estimator_results[best_n]['f1']:.3f})")

---
## Section 6: Experiment — Sample Size (max_samples)

How much data does each tree see? Smaller samples make trees more diverse.

In [ ]:
# === YOUR EXPERIMENT ===
# Try different sample sizes per tree.
# 'auto' means min(256, n_samples)

max_samples_values = ['auto', 64, 128, 256]

# =============================================================================
# Run the experiment
# =============================================================================
print(f"{'max_samples':>12} | {'Precision':>9} | {'Recall':>6} | {'F1':>6}")
print("-" * 45)

samples_results = {}

for s in max_samples_values:
    m = IsolationForest(contamination=0.05, max_samples=s, random_state=42)
    m.fit(X)
    preds = m.predict(X)

    true_bin = (labels == -1).astype(int)
    pred_bin = (preds == -1).astype(int)

    p = precision_score(true_bin, pred_bin, zero_division=0)
    r = recall_score(true_bin, pred_bin, zero_division=0)
    f = f1_score(true_bin, pred_bin, zero_division=0)

    samples_results[str(s)] = {"precision": p, "recall": r, "f1": f}

    print(f"{str(s):>12} | {p:>9.3f} | {r:>6.3f} | {f:>6.3f}")

best_s = max(samples_results, key=lambda k: samples_results[k]['f1'])
print(f"\nBest max_samples by F1 score: {best_s} (F1 = {samples_results[best_s]['f1']:.3f})")

---
## Section 7: Best Configuration

Combine the best values from your experiments and compare against the default.

In [ ]:
# === YOUR EXPERIMENT ===
# Fill in your best values from the experiments above.
# Replace the ___ with the values that gave the best F1 scores.

best_model = IsolationForest(
    contamination=0.05,     # your best value from Section 4
    n_estimators=100,       # your best value from Section 5
    max_samples='auto',     # your best value from Section 6
    random_state=42
)

# =============================================================================
# Compare default vs. best
# =============================================================================

# Default model
default_model = IsolationForest(contamination=0.05, random_state=42)
default_model.fit(X)
default_preds = default_model.predict(X)

# Your best model
best_model.fit(X)
best_preds = best_model.predict(X)

print("DEFAULT MODEL:")
default_results = evaluate_model(labels, default_preds, "Default Isolation Forest")

print("\nYOUR BEST MODEL:")
best_results = evaluate_model(labels, best_preds, "Your Best Configuration")

print(f"\n{'Metric':<12} | {'Default':>8} | {'Best':>8} | {'Change':>8}")
print("-" * 45)
for metric in ['precision', 'recall', 'f1']:
    d = default_results[metric]
    b = best_results[metric]
    change = b - d
    sign = '+' if change >= 0 else ''
    print(f"{metric:<12} | {d:>8.3f} | {b:>8.3f} | {sign}{change:>7.3f}")

---
## Section 8: LOF Comparison

Compare Isolation Forest against Local Outlier Factor (LOF).

LOF uses density estimation instead of isolation. Points in sparse
neighborhoods (far from other points) are flagged as anomalies.

In [ ]:
# =============================================================================
# LOF with default parameters
# =============================================================================

lof_default = LocalOutlierFactor(n_neighbors=20, contamination=0.05)

start = time.time()
lof_preds = lof_default.fit_predict(X)
lof_time = time.time() - start

print("LOF Default (n_neighbors=20):")
lof_default_results = evaluate_model(labels, lof_preds, "LOF (n_neighbors=20)")
print(f"  Training time: {lof_time:.3f}s")

In [ ]:
# === YOUR EXPERIMENT ===
# Try different n_neighbors values for LOF.

n_neighbors_values = [10, 20, 50]

# =============================================================================
# Run the experiment
# =============================================================================
print(f"{'n_neighbors':>12} | {'Precision':>9} | {'Recall':>6} | {'F1':>6} | {'Time':>8}")
print("-" * 55)

lof_results = {}

for n in n_neighbors_values:
    lof = LocalOutlierFactor(n_neighbors=n, contamination=0.05)

    start = time.time()
    preds = lof.fit_predict(X)
    t = time.time() - start

    true_bin = (labels == -1).astype(int)
    pred_bin = (preds == -1).astype(int)

    p = precision_score(true_bin, pred_bin, zero_division=0)
    r = recall_score(true_bin, pred_bin, zero_division=0)
    f = f1_score(true_bin, pred_bin, zero_division=0)

    lof_results[n] = {"precision": p, "recall": r, "f1": f, "time": t}

    print(f"{n:>12} | {p:>9.3f} | {r:>6.3f} | {f:>6.3f} | {t:>7.3f}s")

best_lof_n = max(lof_results, key=lambda k: lof_results[k]['f1'])
print(f"\nBest n_neighbors by F1: {best_lof_n} (F1 = {lof_results[best_lof_n]['f1']:.3f})")

In [ ]:
# =============================================================================
# Head-to-Head Comparison: IF vs. LOF
# =============================================================================

# Use best IF results from Section 7 and best LOF from above
print(f"\n{'='*60}")
print(f"  HEAD-TO-HEAD: Isolation Forest vs. Local Outlier Factor")
print(f"{'='*60}")
print(f"\n{'Metric':<30} | {'IF (Best)':>10} | {'LOF (Best)':>10}")
print("-" * 58)
print(f"{'Precision':<30} | {best_results['precision']:>10.3f} | {lof_results[best_lof_n]['precision']:>10.3f}")
print(f"{'Recall':<30} | {best_results['recall']:>10.3f} | {lof_results[best_lof_n]['recall']:>10.3f}")
print(f"{'F1 Score':<30} | {best_results['f1']:>10.3f} | {lof_results[best_lof_n]['f1']:>10.3f}")
print(f"{'Training Time':<30} | {train_time:>9.3f}s | {lof_results[best_lof_n]['time']:>9.3f}s")
print(f"{'Can score new data one at a time?':<30} | {'Yes':>10} | {'No':>10}")
print(f"{'Needs full dataset present?':<30} | {'No':>10} | {'Yes':>10}")

# Declare winner
if best_results['f1'] >= lof_results[best_lof_n]['f1']:
    print(f"\nOverall winner by F1: Isolation Forest")
else:
    print(f"\nOverall winner by F1: Local Outlier Factor")

print(f"\nNote: Even if LOF has slightly better F1, Isolation Forest is")
print(f"the better choice for real-time IoT because it can score new")
print(f"messages one at a time as they arrive from the MQTT broker.")

---
## Section 9: Optional — One-Class SVM

**This section is optional.** Skip it if you're short on time.

One-Class SVM draws a boundary around normal data in feature space.
Points outside the boundary are anomalies.

In [ ]:
# =============================================================================
# Optional: One-Class SVM Comparison
# =============================================================================

from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

# SVM needs scaled features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train One-Class SVM (nu parameter is similar to contamination)
svm_model = OneClassSVM(nu=0.05, kernel='rbf', gamma='scale')

start = time.time()
svm_model.fit(X_scaled)
svm_time = time.time() - start

svm_preds = svm_model.predict(X_scaled)

print("One-Class SVM:")
svm_results = evaluate_model(labels, svm_preds, "One-Class SVM")
print(f"  Training time: {svm_time:.3f}s")

# Updated comparison table
print(f"\n{'='*70}")
print(f"  THREE-WAY COMPARISON")
print(f"{'='*70}")
print(f"\n{'Metric':<25} | {'IF':>8} | {'LOF':>8} | {'SVM':>8}")
print("-" * 58)
print(f"{'Precision':<25} | {best_results['precision']:>8.3f} | {lof_results[best_lof_n]['precision']:>8.3f} | {svm_results['precision']:>8.3f}")
print(f"{'Recall':<25} | {best_results['recall']:>8.3f} | {lof_results[best_lof_n]['recall']:>8.3f} | {svm_results['recall']:>8.3f}")
print(f"{'F1 Score':<25} | {best_results['f1']:>8.3f} | {lof_results[best_lof_n]['f1']:>8.3f} | {svm_results['f1']:>8.3f}")
print(f"{'Training Time':<25} | {train_time:>7.3f}s | {lof_results[best_lof_n]['time']:>7.3f}s | {svm_time:>7.3f}s")
print(f"{'Real-time scoring?':<25} | {'Yes':>8} | {'No':>8} | {'Yes':>8}")
print(f"{'Needs scaling?':<25} | {'No':>8} | {'No':>8} | {'Yes':>8}")

---
## Section 10: Export Model

Save your trained Isolation Forest model to a file.
You'll download this and use it in `subscriber_dashboard_ai.py`.

In [ ]:
# =============================================================================
# Export the Best Isolation Forest Model
# =============================================================================

import os
import joblib

# Use the best model from Section 7
# (If you changed the parameters there, this exports YOUR best model)
export_model = best_model

# Save to file
model_path = 'anomaly_model.joblib'
joblib.dump(export_model, model_path)

print(f"Model saved to: {model_path}")
print(f"File size: {os.path.getsize(model_path) / 1024:.1f} KB")
print(f"\nTo use this model:")
print(f"  1. Download {model_path} from Colab (Files panel > right-click > Download)")
print(f"  2. Place it in your Project 8 directory")
print(f"  3. Run subscriber_dashboard_ai.py — it will load the model automatically")

# Quick verification: load it back and score one point
loaded_model = joblib.load(model_path)
test_point = np.array([[60.0, 50.0, 45.0]])  # normal reading
test_pred = loaded_model.predict(test_point)
test_score = loaded_model.decision_function(test_point)
print(f"\nVerification — test point [60 PSI, 50 LPM, 45%]:")
print(f"  Prediction: {'Normal' if test_pred[0] == 1 else 'ANOMALY'}")
print(f"  Score: {test_score[0]:.3f}")

In [ ]:
# =============================================================================
# Download Helper (for Google Colab)
# =============================================================================

try:
    from google.colab import files
    files.download('anomaly_model.joblib')
    print("Download started! Check your browser's download folder.")
except ImportError:
    print("Not running in Colab — find anomaly_model.joblib in your working directory.")
except Exception as e:
    print(f"Auto-download failed: {e}")
    print("Manually download: Files panel (left sidebar) > anomaly_model.joblib > Download")